## Detailed Explanation

1. Representation
- n: number of vertices (0..n-1)
- edges: list of (u,v,w)
- Adjacency list stores (neighbor, weight)
- State = (current node, visited bitmask). Bitmask has bit i = 1 if node i visited.

2. A* Components
- g = cost accumulated along the partial path (sum of edge weights traversed so far)
- h = admissible heuristic lower bound on remaining cost
- f = g + h (priority key in the min-heap)

3. Heuristic h(node, mask)
- remaining_mask = nodes not yet visited
- MST over remaining nodes: any Hamiltonian completion must at least connect all remaining nodes, so MST cost is a lower bound
- Plus cheapest edge from current node to any remaining node (must leave current node to enter remaining set)
- (We do NOT add a return/closure edge since we solve Hamiltonian path, not necessarily a cycle)

4. MST Computation (mst_cost_subset)
- Extract all edges whose endpoints are fully inside the subset
- Run Kruskal (sorting edges) with DSU to accumulate minimal spanning tree cost
- Cache by subset bitmask with lru_cache to avoid recomputation

5. Search Loop
- Pop lowest f state
- If all nodes visited (mask == (1<<n)-1), path is complete
- Expand by trying every unvisited neighbor; update g, recompute h, push new state if better

6. Correctness & Admissibility
- Heuristic never overestimates: true remaining cost ≥ (MST over remaining) + (must connect current to remaining)
- Monotonic (consistent) in practice because edge weights ≥ 0 and MST lower bound respects triangle inequality structurally

7. Complexity
- Worst-case still O(n * 2^n) states (like DP Held-Karp) but pruning via heuristic reduces expansions
- MST cost caching reduces repeated O(E log E) work

8. Limitations / Variations
- For Hamiltonian cycle (TSP style), you would also add cheapest edge from any remaining node back to a fixed start node in h
- For very large n, switch to approximate heuristics (e.g., sum of two smallest incident edges per remaining node / 2)

9. Edge Cases
- Disconnected remaining subset ⇒ MST = inf ⇒ heuristic = inf, pruning that branch
- Graphs with a node of degree 0 (n>1) immediately unsatisfiable

10. Output
- Returns (cost, path list). If cost = inf, no Hamiltonian path exists.


In [9]:
from dataclasses import dataclass, field
import heapq
from functools import lru_cache
from typing import List, Tuple, Dict, Optional

@dataclass(order=True)
class PrioritizedState:
    f: float
    g: float = field(compare=False)
    node: int = field(compare=False)
    mask: int = field(compare=False)
    path: Tuple[int,...] = field(compare=False)

class DSU:
    def __init__(self, n):
        self.p = list(range(n))
        self.r = [0]*n
    def find(self, x):
        while self.p[x] != x:
            self.p[x] = self.p[self.p[x]]
            x = self.p[x]
        return x
    def union(self, a,b):
        ra, rb = self.find(a), self.find(b)
        if ra==rb: return False
        if self.r[ra] < self.r[rb]:
            ra, rb = rb, ra
        self.p[rb] = ra
        if self.r[ra] == self.r[rb]:
            self.r[ra]+=1
        return True

class AStarHamiltonian:
    def __init__(self, n: int, edges: List[Tuple[int,int,float]]):
        self.n = n
        self.edges = edges
        self.adj: List[List[Tuple[int,float]]] = [[] for _ in range(n)]
        for u,v,w in edges:
            self.adj[u].append((v,w))
            self.adj[v].append((u,w))
        # Preprocess cheapest edge from any node to any other for fallback
        self.global_min_edge = min((w for *_, w in edges), default=0)

    @lru_cache(maxsize=None)
    def mst_cost_subset(self, mask: int) -> float:
        """Compute MST cost of nodes indicated in mask using Kruskal. Nodes are 0..n-1."""
        nodes = [i for i in range(self.n) if mask & (1<<i)]
        k = len(nodes)
        if k <= 1:
            return 0.0
        # Map original index -> compressed 0..k-1
        idx = {v:i for i,v in enumerate(nodes)}
        # Collect edges internal to subset
        sub_edges = []
        for u,v,w in self.edges:
            if (mask & (1<<u)) and (mask & (1<<v)):
                sub_edges.append((w,u,v))
        sub_edges.sort()
        dsu = DSU(k)
        total = 0.0
        used = 0
        for w,u,v in sub_edges:
            if dsu.union(idx[u], idx[v]):
                total += w
                used += 1
                if used == k-1:
                    break
        if used != k-1:
            return float('inf')  # disconnected subset
        return total

    def heuristic(self, node: int, mask: int) -> float:
        # Remaining nodes (excluding current node if not yet counted) are those not in mask
        # mask already includes visited nodes
        remaining_mask = ((1<<self.n)-1) ^ mask
        if remaining_mask == 0:
            return 0.0
        # Cost to connect remaining nodes minimally
        mst_cost = self.mst_cost_subset(remaining_mask)
        # Cheapest edge from current node to any remaining node
        connect_cost = min((w for v,w in self.adj[node] if remaining_mask & (1<<v)), default=0.0)
        # (Optional) Lower bound could also include an exit edge if you enforce path end constraints
        return mst_cost + connect_cost

    def solve(self, start: int = 0) -> Tuple[float, List[int]]:
        n = self.n
        start_mask = 1<<start
        pq: List[PrioritizedState] = []
        start_state = PrioritizedState(
            f=0 + self.heuristic(start, start_mask),
            g=0.0,
            node=start,
            mask=start_mask,
            path=(start,)
        )
        heapq.heappush(pq, start_state)
        # Best g for (node,mask)
        best: Dict[Tuple[int,int], float] = {(start, start_mask): 0.0}

        while pq:
            st = heapq.heappop(pq)
            if st.mask == (1<<n)-1:
                # Found full path
                return st.g, list(st.path)
            # If outdated
            if st.g > best.get((st.node, st.mask), float('inf')):
                continue
            for v,w in self.adj[st.node]:
                bit = 1<<v
                if st.mask & bit:
                    continue  # already visited
                g2 = st.g + w
                mask2 = st.mask | bit
                key = (v, mask2)
                if g2 < best.get(key, float('inf')):
                    best[key] = g2
                    h2 = self.heuristic(v, mask2)
                    heapq.heappush(pq, PrioritizedState(g2 + h2, g2, v, mask2, st.path + (v,)))
        return float('inf'), []  # No Hamiltonian path

# Example usage
""" n = 5
edges = [
    (0,1,2),(0,2,9),(0,3,3),(0,4,8),
    (1,2,6),(1,3,4),(1,4,7),
    (2,3,5),(2,4,1),
    (3,4,2)
] """

""" n = 5
edges = [
    (i,j,1+(i+j)%7) for i in range(5) for j in range(i+1,5)
] """

n = 5
edges = [
    (0,1,2),(1,2,2),(2,3,2),(3,4,2),(0,2,5),(1,3,5)
]
solver = AStarHamiltonian(n, edges)
cost, path = solver.solve(start=0)
print("Cost:", cost)
print("Path:", path)

Cost: 8.0
Path: [0, 1, 2, 3, 4]


In [8]:
# Additional test cases (choose one by uncommenting)
# 1. Simple path (line) graph of 4 nodes (unique Hamiltonian path)
# n = 4
# edges = [
#     (0,1,1),(1,2,2),(2,3,3)
# ]

# 2. Star graph (no Hamiltonian path unless n <= 2)
# n = 5
# edges = [
#     (0,1,1),(0,2,1),(0,3,1),(0,4,1)
# ]

# 3. Complete graph K5 (many Hamiltonian paths)
# n = 5
# edges = [
#     (i,j,1+(i+j)%7) for i in range(5) for j in range(i+1,5)
# ]

# 4. Disconnected graph (no path)
# n = 4
# edges = [
#     (0,1,2),(2,3,2)
# ]

# 5. Graph with multiple equal-cost Hamiltonian paths
# n = 4
# edges = [
#     (0,1,1),(1,2,1),(2,3,1),(0,2,2),(1,3,2),(0,3,3)
# ]

# 6. Sparse graph where only one ordering works
# n = 5
# edges = [
#     (0,1,2),(1,2,2),(2,3,2),(3,4,2),(0,2,5),(1,3,5)
# ]

# Utility: build solver, run, print
if __name__ == '__main__':
    print('\nRunning main example already defined above:')
    print('Cost:', cost)
    print('Path:', path)



Running main example already defined above:
Cost: 13.0
Path: [0, 1, 2, 3, 4]


## Backtracking (DFS) Approach for Weighted Hamiltonian Path

This section implements a classic depth-first backtracking search to find a minimum-cost Hamiltonian path in a weighted, undirected graph.

Key points:
- Tries every possible ordering (worst-case O(n!)).
- Prunes partial paths whose current cost already exceeds the best cost found so far (branch & bound).
- Works well only for small n (≈ ≤ 12 depending on edge density & pruning efficiency).
- Unlike the A* version, this does not use a heuristic—just raw backtracking with pruning.

Data structures:
- adj[u] = list of (v, w) pairs.
- visited[] boolean list.
- current_path list collects nodes in the current recursion chain.
- best_path / best_cost track the global optimum.

Enhancements included:
- Neighbor ordering: sort adjacency lists by edge weight (lighter edges explored first) to reach good solutions earlier → stronger pruning.
- Early exit if a Hamiltonian path with cost equal to theoretical lower bound (sum of n-1 lightest distinct incident edges forming a chain) is reached (optional placeholder shown).


In [10]:
from math import inf
from typing import List, Tuple

def hamiltonian_path_backtracking(n: int, edges: List[Tuple[int,int,float]], start: int = None):
    # Build adjacency list
    adj: List[List[Tuple[int,float]]] = [[] for _ in range(n)]
    for u,v,w in edges:
        adj[u].append((v,w))
        adj[v].append((u,w))
    # Sort neighbors by ascending weight for better pruning
    for u in range(n):
        adj[u].sort(key=lambda x: x[1])

    visited = [False]*n
    best_cost = inf
    best_path: List[int] = []
    path: List[int] = []

    def dfs(u: int, depth: int, cost_so_far: float):
        nonlocal best_cost, best_path
        # Prune if already worse than best
        if cost_so_far >= best_cost:
            return
        if depth == n:  # Found Hamiltonian path
            if cost_so_far < best_cost:
                best_cost = cost_so_far
                best_path = path.copy()
            return
        for v,w in adj[u]:
            if not visited[v]:
                visited[v] = True
                path.append(v)
                dfs(v, depth+1, cost_so_far + w)
                path.pop()
                visited[v] = False

    starts = [start] if start is not None else list(range(n))
    for s in starts:
        visited = [False]*n
        visited[s] = True
        path = [s]
        dfs(s, 1, 0.0)

    return best_cost, best_path

# Example weighted graphs to test backtracking
# (Reuse edge sets from above or define new ones)
examples = [
    (5, [  # Example 1: same as earlier A* example variant
        (0,1,2),(0,2,9),(0,3,3),(0,4,8),
        (1,2,6),(1,3,4),(1,4,7),
        (2,3,5),(2,4,1),
        (3,4,2)
    ]),
    (5, [  # Example 2: sparse chain with extra chords
        (0,1,2),(1,2,2),(2,3,2),(3,4,2),(0,2,5),(1,3,5)
    ]),
    (4, [  # Example 3: small complete graph
        (0,1,3),(0,2,2),(0,3,4),(1,2,1),(1,3,5),(2,3,6)
    ]),
]

for idx,(nG, eG) in enumerate(examples, start=1):
    cost_bt, path_bt = hamiltonian_path_backtracking(nG, eG)
    print(f"Example {idx}: n={nG}, cost={cost_bt}, path={path_bt}")


Example 1: n=5, cost=8.0, path=[1, 0, 3, 4, 2]
Example 2: n=5, cost=8.0, path=[0, 1, 2, 3, 4]
Example 3: n=4, cost=7.0, path=[1, 2, 0, 3]
